In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict
import re

pd.set_option('display.max_colwidth' , -1)

import sys
import re
import os
from pathlib import Path
from collections import namedtuple
import numpy as np

In [3]:
import xml.etree.ElementTree as ET
english = ET.parse('../data/xml_files/English_restaurants.xml')
root = english.getroot()
labeled_reviews = []
for review in root.findall("Review"):
    rid = review.get('rid')
    if review.find("sentences"):
        for sentence in review.find("sentences").findall('sentence'): 
            if sentence.find('Opinions'):
                #print(sentence[0].text)
                entry= {} ;   aspects = [] ;  polarities = []
                for opinion in sentence.find('Opinions').findall('Opinion'):
                    aspects.append(opinion.get('category'))
                    polarities.append(opinion.get('polarity'))
            
                entry['review_id'] = rid#.copy()
                entry["text"] = sentence[0].text
                entry["aspects"] , entry['polarities']  =  aspects , polarities
                labeled_reviews.append(entry)
            
eng_multi_aspects = pd.DataFrame(labeled_reviews)
print("there are",len(labeled_reviews),"reviews in this training set")

#eng_multi_aspects.to_csv('../data/English_restaurants.csv' , index = False)

there are 1708 reviews in this training set


In [4]:
df_complete_review = eng_multi_aspects.groupby(['review_id'])['text'].apply(lambda x: ' '.join(x) ).reset_index()

In [5]:
df_complete_review.tail(2)

,review_id,text
347,Z#7,"This place doesn't make any sense Just want to warn you all - don't waste your time and money. This place has totally weird decor, stairs going up with mirrored walls - I am surprised how no one yet broke their head or fall off the stairs - mirrored walls make you dizzy and delusional... This place is not inviting and the food is totally weird. The concept of japanese tapas is newly created and clearly doesn't work. The food they serve is not comforting, not appetizing and uncooked."
348,Z#9,"short and sweet – seating is great:it's romantic,cozy and private. The boths are not as small as some of the reviews make them out to look they're perfect for 2 people. The service was extremely fast and attentive(thanks to the service button on your table) but I barely understood 1 word when the waiter took our order. The food was ok and fair nothing to go crazy. Over all the looks of the place exceeds the actual meals. So what you really end up paying for is the restaurant not the food. Will prob. not return but it is a great dinning experience to try atleast once."


In [6]:
# #from neuralcoref import Coref
# import en_core_web_sm
# spacy = en_core_web_sm.load()

In [8]:
import neuralcoref

/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


In [ ]:

import spacy#en_core_web_sm
nlp = spacy.load('en_core_web_sm')#en_core_web_sm.load()

In [ ]:
neuralcoref.add_to_pipe(nlp)

def replace_pronouns(text):
    result = doc._.coref_resolved  
    return result 

In [ ]:
doc = nlp(u'My sister has a dog. She loves him.')

In [6]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

def parse_sentence(line):
    lmtzr = WordNetLemmatizer()
    stop = stopwords.words('english')
    exclude = set(string.punctuation) 
    text_token = CountVectorizer().build_tokenizer()(line.lower())
    text_pos = nltk.pos_tag(text_token)
    text_nn = check_nouns(text_pos)
    text_rmstop = [i for i in text_nn if i not in stop]
    text_rmstop = [i for i in text_rmstop if i not in exclude]
    text_stem = [lmtzr.lemmatize(w) for w in text_rmstop]
    return ' '.join(text_stem)

In [9]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import codecs
import nltk
#nltk.download('averaged_perceptron_tagger')

def check_nouns(x):
    res = []
    for ele in x:
        if 'NN' in  ele[1]: 
            res.append(ele[0])
    return res


In [7]:
import gensim
from gensim import corpora
restaurant_data = '/home/FRACTAL/swati.tiwari/kaggle/yelp/Unsupervised-Aspect-Extraction-master/datasets/restaurant_old/train.txt'

In [10]:
file = open(restaurant_data , 'r')  
rest_280k = file.readlines()
#doc_clean = [clean(doc).split() for doc in rest_280k] 
lemmatized_280k = [parse_sentence(x) for x in rest_280k]

In [19]:
def display_text(H , W , feature_names , documents , no_top_words , no_top_documents):
    for topic_id , topic in enumerate(H):
        print('Topic' , topic_id , " ".join(feature_names[i] for i in topic.argsort()[: -no_top_words -1 : -1] ))
        #print('topic' , topic_id , ' '.join([feature_names[i] for i in topic.argsort()[-no_top_words:]]))
        top_doc_indices = np.argsort(W[: , topic_id])[:: -1][0 : no_top_documents]
#         for doc in top_doc_indices:
#             print(documents[doc] , end = '')

In [14]:
no_features = 1000
no_topics = 10
no_top_words = 10
no_top_documents = 5

In [15]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df = 0.95 , min_df = 3, max_features= no_features , stop_words= 'english' , )
tfidf = tfidf_vectorizer.fit_transform(lemmatized_280k)

tf_vectorizer = CountVectorizer(max_df = 0.95 , min_df = 3 , max_features= no_features , stop_words= 'english' )
tf = tf_vectorizer.fit_transform(lemmatized_280k)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()
tf_feature_names = tf_vectorizer.get_feature_names()

In [16]:
nmf_model = NMF(n_components=no_topics , random_state=1 , alpha=0.1 , l1_ratio= 0.5 , init = 'nndsvd').fit(tfidf)
nmf_W = nmf_model.transform(tfidf)
nmf_H = nmf_model.components_
display_text(nmf_H , nmf_W , tfidf_feature_names ,rest_280k , no_top_words , no_top_documents)

Topic 0 food atmosphere quality drink portion ambience decor ambiance wine taste
The food is outstanding!!!!!! You will not regret this choose
Overall, I thought the food was OK, at best
Food is equally great
The food is always excellent and different
The food was definitely good
Topic 1 place date people city neighborhood review sushi year thing pizza
This place is ok
I just love this place
I would not recommend this place to my enemies
I've dined in some very fine places, and this was not one of them
I love this place
Topic 2 service atmosphere ambience decor waiter ambiance customer meal waitress notch
The service is friendly and attentive without being forced, atmosphere is relaxing
And the service remains stellar
I was a little disappointed by the service too
The service was friendly but I would never go back
The service was excellent
Topic 3 restaurant city york nyc week neighborhood year area review people
All restaurants should follow their lead
All she could do was tell me how

In [18]:
lda_model = LatentDirichletAllocation(n_topics = no_topics , max_iter = 10 , learning_method = 'online' , learning_offset = 50 , random_state =1   , doc_topic_prior = 0.01 *no_topics , topic_word_prior = 0.01 *no_topics ).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

display_text(lda_H , lda_W , tf_feature_names ,rest_280k , no_top_words , no_top_documents)

/home/FRACTAL/swati.tiwari/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0 dinner experience pizza friend wine area week brunch money date
Went to Jane with a group of 10 friends for a birthday dinner last week
Last week we had dinner there for my wife's birthday
I had the misfortune of taking my new husband to Savoy for his birthday dinner last week
my girlfriend and I had had brunch here a few weeks prior and enjoyed it so very much we decided to come back and try dinner
I recently brought my husband here on a Monday evening for his birthday dinner
Topic 1 time minute salad lunch burger pasta server cheese fry house
Trust me, you must try this burger, I went to DB for lunch today and the burger was enjoyable and quite different but it was not a real burger
A few minutes later, the guy showed up with cold burger and fries
Next time we'll stick to the Calamari and a house salad
My favorite is their house salad with the house dressing
i recommend their burger with goat cheese on it - phenominal (fries are really good too)
Topic 2 people waitress neighb

##  Can use bi grams tri grams features , and only keep verb and noun phrase by using POS tagging## NMF is surely performing better

In [20]:
## remove word food as stop word as it appears in all the documents.
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''


In [38]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [31]:
arr =  nltk.pos_tag(nltk.word_tokenize(rest_280k[10]))

In [107]:
def parse_sentence(line):
    lmtzr = WordNetLemmatizer()
    stop = stopwords.words('english') + ['food' , '' , 'un' , 've'] +  ['friend', 'go' , 'back']
    exclude = set(string.punctuation) 
    text_token = CountVectorizer().build_tokenizer()(line.lower())
    text_pos = nltk.pos_tag(text_token)
    text_selected = [ lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) if get_wordnet_pos(el[1]) != '' else '' for el in text_pos]
    #text_nn = check_nouns(text_pos)
    text_rmstop = [i for i in text_selected if i not in stop]
    text_rmstop = [i for i in text_rmstop if i not in exclude]
    text_stem = [lmtzr.lemmatize(w) for w in text_rmstop]
    return text_stem#' '.join(text_stem)

In [108]:
processed_txt = [parse_sentence(x) for x in rest_280k]

In [73]:
from gensim.models import Phrases

In [109]:
tokens = processed_txt
bigram_model = Phrases(tokens , threshold = 5 , min_count = 10 )
#trigram_model = Phrases(bigram_model[tokens], min_count=2 , threshold = 5)
processed_txt_bigrams =  list(bigram_model[tokens]) #List(trigram_model[bigram_model[tokens]])

In [110]:
processed_txt_bigrams2 = [' '.join(x) for x in processed_txt_bigrams]

In [111]:
processed_txt_bigrams2

['jeollado roll sometimes price variety menu',
 'roll tiny order anyway often get order wrong stray menu',
 'money dependable fun place get sushi bring share roll',
 '',
 'place great_deal price give',
 'crab roll make real crab imitation crab',
 'also great unagi bim bim bap order',
 'want little_bit korean combine little_bit japanese',
 'place casual fancy',
 'short cash big_group starve place',
 'okay delicate delectible absolutely satisfying sake cheap',
 'portion pretty amazing especially amy roll damn thats lot',
 'jeollado seaweed_salad sometimes get take somewhere_else get salad healthy fill',
 'great place refined',
 'oh also pretty_good scene lot people look',
 'nice place cheap place jeollado probably_best cheap sushi new_york city',
 'jeollado house roll definitely popular draw',
 'damn_good',
 'try',
 'jeollado bunch time fish nearly always_fresh',
 'admit stumble couple fresh day rare',
 'wait_staff nice kind inattentive',
 'atmosphere cafeteria romantic',
 'however casua

In [112]:
tfidf_vectorizer = TfidfVectorizer(max_df = 0.95 , min_df = 3, max_features= no_features , stop_words= 'english' , )
tfidf = tfidf_vectorizer.fit_transform(processed_txt_bigrams2)

tf_vectorizer = CountVectorizer(max_df = 0.95 , min_df = 3 , max_features= no_features , stop_words= 'english' )
tf = tf_vectorizer.fit_transform(processed_txt_bigrams2)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()
tf_feature_names = tf_vectorizer.get_feature_names()

In [113]:
nmf_model = NMF(n_components=14 , random_state=1 , alpha=0.1 , l1_ratio= 0.5 , init = 'nndsvd').fit(tfidf)
nmf_W = nmf_model.transform(tfidf)
nmf_H = nmf_model.components_
display_text(nmf_H , nmf_W , tfidf_feature_names ,rest_280k , no_top_words , no_top_documents)

Topic 0 great atmosphere price experience drink meal music date ambience wine
Topic 1 good really price thing pizza quite think look sushi overall
Topic 2 place recommend nice look eat want date really small highly_recommend
Topic 3 service excellent friendly bad atmosphere attentive wonderful nice ambience outstanding
Topic 4 restaurant favorite recommend nyc city new_york highly_recommend new experience small
Topic 5 order come table delicious drink menu wait make dinner waiter
Topic 6 try definitely dish want dessert worth menu new chicken appetizer
Topic 7 love atmosphere decor really pizza especially husband boyfriend sushi wife
Topic 8 best city nyc new_york sushi town steak pizza burger meal
Topic 9 time eat wonderful couple twice visit reservation week enjoy lunch


In [116]:

lda_model = LatentDirichletAllocation(n_topics = no_topics , max_iter = 10 , learning_method = 'online' , learning_offset = 50 , random_state =1   , doc_topic_prior = 0.01 *no_topics , topic_word_prior = 0.01 *no_topics ).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

display_text(lda_H , lda_W , tf_feature_names ,rest_280k , no_top_words , no_top_documents)

/home/FRACTAL/swati.tiwari/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0 order eat menu dish meal say drink know people ask
Topic 1 try make recommend appetizer special salad chicken sauce burger delicious
Topic 2 time table wait staff bar steak definitely seat small sit
Topic 3 really love dinner think little wonderful good enjoy tell decor
Topic 4 great place taste experience night tasty like use spot family
Topic 5 place good nice restaurant look city best sushi fresh nyc
Topic 6 pizza dessert delicious way entree lunch reservation pasta neighborhood quite
Topic 7 price best amaze expect especially fun bit highly_recommend romantic start
Topic 8 service excellent atmosphere good serve friendly wine leave perfect quality
Topic 9 restaurant come waiter want bad day time italian visit party


In [121]:
eng_multi_aspects[['text' , 'aspects']].tail()

,text,aspects
1703,"I can't believe that it was, but please put the bag down before delivering food!",[SERVICE#GENERAL]
1704,"The waitress came to check in on us every few minutes, and began to clear the plates while half of us were still eating (a big pet peeve of mine that happens almost everywhere, so I try to ignore it).",[SERVICE#GENERAL]
1705,"I couldn't ignore the fact that she reach over the plate of one of my friends, who was in mid bite, to clear the table.",[SERVICE#GENERAL]
1706,"She then put the check down without asking if we were done, and came to check on the bill every two minutes, even though we were one of three occupied tables.",[SERVICE#GENERAL]
1707,"I wish I could like this place more, and I wish someone would retrain the staff.","[RESTAURANT#GENERAL, SERVICE#GENERAL]"


In [122]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [124]:
cod = nlp(eng_multi_aspects['text'].loc[1707])

In [131]:
for ent in cod.ents:
    print(ent)